In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # 주문 관련 초기화
    if (g_objCpTrade.TradeInit(0) != 0):
        print("주문 초기화 실패")
        return False
 
    return True
 
 
# CpEvent: 실시간 이벤트 수신 클래스
class CpEvent:
    def set_params(self, client, name, caller):
        self.client = client  # CP 실시간 통신 object
        self.name = name  # 서비스가 다른 이벤트를 구분하기 위한 이름
        self.caller = caller  # callback 을 위해 보관
 
        # 구분값 : 텍스트로 변경하기 위해 딕셔너리 이용
        self.dicflag12 = {'1': '매도', '2': '매수'}
        self.dicflag14 = {'1': '체결', '2': '확인', '3': '거부', '4':'접수'}
        self.dicflag15 = {'00': '현금', '01': '유통융자', '02': '자기융자', '03': '유통대주',
                          '04':'자기대주', '05':'주식담보대출', '07':'채권담보대출',
                          '06':'매입담보대출', '08':'플러스론',
                          '13':'자기대용융자', '15':'유통대용융자'}
        self.dicflag16 = {'1': '정상주문', '2': '정정주문', '3': '취소주문'}
        self.dicflag17 = {'1': '현금', '2': '신용', '3': '선물대용', '4': '공매도'}
        self.dicflag18 = {'01': '보통', '02': '임의', '03':'시장가', '05': '조건부지정가'}
        self.dicflag19 = {'0': '없음', '1': 'IOC', '2': 'FOK'}
 
 
    def OnReceived(self):
#        print(self.name)
        # 실시간 처리 - 주문체결
        if self.name == 'conclution':
            # 주문 체결 실시간 업데이트
            conc = {}
 
            # 체결 플래그
            conc['체결플래그'] = self.dicflag14[self.client.GetHeaderValue(14)]
 
            conc['주문번호'] = self.client.GetHeaderValue(5)  # 주문번호
            conc['주문수량'] = self.client.GetHeaderValue(3)  # 주문/체결 수량
            conc['주문가격'] = self.client.GetHeaderValue(4)  # 주문/체결 가격
            conc['원주문'] = self.client.GetHeaderValue(6)
            conc['종목코드'] = self.client.GetHeaderValue(9)  # 종목코드
            conc['종목명'] = g_objCodeMgr.CodeToName(conc['종목코드'])
 
            conc['매수매도'] = self.dicflag12[self.client.GetHeaderValue(12)]
 
            flag15  = self.client.GetHeaderValue(15) # 신용대출구분코드
            if (flag15 in self.dicflag15):
                conc['신용대출'] = self.dicflag15[flag15]
            else:
                conc['신용대출'] = '기타'
 
            conc['정정취소'] = self.dicflag16[self.client.GetHeaderValue(16)]
            conc['현금신용'] = self.dicflag17[self.client.GetHeaderValue(17)]
            conc['주문조건'] = self.dicflag19[self.client.GetHeaderValue(19)]
 
            conc['체결기준잔고수량'] = self.client.GetHeaderValue(23)
            loandate = self.client.GetHeaderValue(20)
            if (loandate == 0):
                conc['대출일'] = ''
            else:
                conc['대출일'] = str(loandate)
            flag18 = self.client.GetHeaderValue(18)
            if (flag18 in self.dicflag18):
                conc['주문호가구분'] = self.dicflag18[flag18]
            else:
                conc['주문호가구분'] = '기타'
 
            conc['장부가'] = self.client.GetHeaderValue(21)
            conc['매도가능'] = self.client.GetHeaderValue(22)
 
            print(conc)
            self.caller.updateJangoCont(conc)
 
            return
 
 
class CpPublish:
    def __init__(self, name, serviceID):
        self.name = name
        self.obj = win32com.client.Dispatch(serviceID)
        self.bIsSB = False
 
    def Subscribe(self, var, caller):
        if self.bIsSB:
            self.Unsubscribe()
 
        if (len(var) > 0):
            self.obj.SetInputValue(0, var)
 
        handler = win32com.client.WithEvents(self.obj, CpEvent)
        handler.set_params(self.obj, self.name, caller)
        self.obj.Subscribe()
        self.bIsSB = True
 
    def Unsubscribe(self):
        if self.bIsSB:
            self.obj.Unsubscribe()
        self.bIsSB = False
 
# CpPBConclusion: 실시간 주문 체결 수신 클래그
class CpPBConclusion(CpPublish):
    def __init__(self):
        super().__init__('conclution', 'DsCbo1.CpConclusion')
 
# Cp6033 : 주식 잔고 조회
class Cp6033:
    def __init__(self):
        acc = g_objCpTrade.AccountNumber[0]  # 계좌번호
        accFlag = g_objCpTrade.GoodsList(acc, 1)  # 주식상품 구분
        print(acc, accFlag[0])
 
        self.objRq = win32com.client.Dispatch("CpTrade.CpTd6033")
        self.objRq.SetInputValue(0, acc)  # 계좌번호
        self.objRq.SetInputValue(1, accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        self.objRq.SetInputValue(2, 50)  # 요청 건수(최대 50)
 
    # 실제적인 6033 통신 처리
    def requestJango(self, caller):
        while True:
            ret = self.objRq.BlockRequest()
            if ret == 4 :
                remainTime = g_objCpStatus.LimitRequestRemainTime
                print('연속조회 제한 오류, 남은 시간', remainTime)
                return False
            # 통신 및 통신 에러 처리
            rqStatus = self.objRq.GetDibStatus()
            rqRet = self.objRq.GetDibMsg1()
            print("통신상태", rqStatus, rqRet)
            if rqStatus != 0:
                return False
 
            cnt = self.objRq.GetHeaderValue(7)
            print(cnt)
 
            for i in range(cnt):
                item = {}
                code = self.objRq.GetDataValue(12, i)  # 종목코드
                item['종목코드'] = code
                item['종목명'] = self.objRq.GetDataValue(0, i)  # 종목명
                item['현금신용'] = self.objRq.GetDataValue(1, i)  # 신용구분
                print(code, '현금신용', item['현금신용'])
                item['대출일'] = self.objRq.GetDataValue(2, i)  # 대출일
                item['잔고수량'] = self.objRq.GetDataValue(7, i)  # 체결잔고수량
                item['매도가능'] = self.objRq.GetDataValue(15, i)
                item['장부가'] = self.objRq.GetDataValue(17, i)  # 체결장부단가
                # 매입금액 = 장부가 * 잔고수량
                item['매입금액'] = item['장부가'] * item['잔고수량']
 
                # 잔고 추가
                caller.jangoData[code] = item
 
                if len(caller.jangoData) >= 200:  # 최대 200 종목만,
                    break
 
            if len(caller.jangoData) >= 200:
                break
            if (self.objRq.Continue == False):
                break
        return True
 
 
# 현재가 - 한종목 통신
class CpRPCurrentPrice:
    def __init__(self, caller):
        self.caller = caller
        self.objStockMst = win32com.client.Dispatch('DsCbo1.StockMst')
        return
 
    def Request(self, code):
        self.caller.curData[code] = {}
        self.objStockMst.SetInputValue(0, code)
        ret = self.objStockMst.BlockRequest()
        if self.objStockMst.GetDibStatus() != 0:
            print('통신상태', self.objStockMst.GetDibStatus(), self.objStockMst.GetDibMsg1())
            return False
 
        item = {}
        item['code'] = code
        #caller.curData['종목명'] = g_objCodeMgr.CodeToName(code)
        item['cur'] = self.objStockMst.GetHeaderValue(11)  # 종가
        item['diff'] = self.objStockMst.GetHeaderValue(12)  # 전일대비
        item['vol'] = self.objStockMst.GetHeaderValue(18)  # 거래량
 
        # 10차호가
        for i in range(10):
            key1 = 'offer%d' % (i + 1)
            key2 = 'bid%d' % (i + 1)
            item[key1] = (self.objStockMst.GetDataValue(0, i))  # 매도호가
            item[key2] = (self.objStockMst.GetDataValue(1, i))  # 매수호가
 
        self.caller.curData[code] = item
        return True
 
 
 
 
# 주식 주문 처리
class CpRPOrder:
    def __init__(self, caller):
        self.caller = caller
        self.acc = g_objCpTrade.AccountNumber[0]  # 계좌번호
        self.accFlag = g_objCpTrade.GoodsList(self.acc, 1)  # 주식상품 구분
        print(self.acc, self.accFlag[0])
        self.objOrder = win32com.client.Dispatch("CpTrade.CpTd0311")  # 매수
 
    def buyOrder(self, code, price, amount):
        # 주식 매수 주문
        print("신규 매수", code, price, amount)
 
        self.objOrder.SetInputValue(0, "2")  # 2: 매수
        self.objOrder.SetInputValue(1, self.acc)  # 계좌번호
        self.objOrder.SetInputValue(2, self.accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        self.objOrder.SetInputValue(3, code)  # 종목코드
        self.objOrder.SetInputValue(4, amount)  # 매수수량
        self.objOrder.SetInputValue(5, price)  # 주문단가
        self.objOrder.SetInputValue(7, "0")  # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
        self.objOrder.SetInputValue(8, "01")  # 주문호가 구분코드 - 01: 보통
 
        # 매수 주문 요청
        ret = self.objOrder.BlockRequest()
        if ret == 4:
            remainTime = g_objCpStatus.LimitRequestRemainTime
            print('주의: 주문 연속 통신 제한에 걸렸음. 대기해서 주문할 지 여부 판단이 필요 남은 시간', remainTime)
            return False
 
        rqStatus = self.objOrder.GetDibStatus()
        rqRet = self.objOrder.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False
 
        return True
 
 
    def sellOrder(self, code, price, amount):
        # 주식 매도 주문
        print("신규 매도", code, price, amount)
 
        self.objOrder.SetInputValue(0, "1")  # 1: 매도
        self.objOrder.SetInputValue(1, self.acc)  # 계좌번호
        self.objOrder.SetInputValue(2, self.accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        self.objOrder.SetInputValue(3, code)  # 종목코드
        self.objOrder.SetInputValue(4, amount)  # 매수수량
        self.objOrder.SetInputValue(5, price)  # 주문단가
        self.objOrder.SetInputValue(7, "0")  # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
        self.objOrder.SetInputValue(8, "01")  # 주문호가 구분코드 - 01: 보통
 
        # 매도 주문 요청
        ret = self.objOrder.BlockRequest()
        if ret == 4:
            remainTime = g_objCpStatus.LimitRequestRemainTime
            print('주의: 주문 연속 통신 제한에 걸렸음. 대기해서 주문할 지 여부 판단이 필요 남은 시간', remainTime)
            return False
 
        rqStatus = self.objOrder.GetDibStatus()
        rqRet = self.objOrder.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False
 
        return True
 
 
 
class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
 
        # plus 상태 체크
        if InitPlusCheck() == False:
            exit()
 
        self.setWindowTitle("주식 잔고 일괄매도 예제")
 
        # 6033 잔고 object
        self.obj6033 = Cp6033()
        self.jangoData = {}
        self.objConclusion = CpPBConclusion()
 
        self.objRpCur = CpRPCurrentPrice(self)
        self.curData = {}
 
        self.objRpOrder = CpRPOrder(self)
 
        nH = 20
        btnSellAll = QPushButton('전체매도', self)
        btnSellAll.move(20, nH)
        btnSellAll.clicked.connect(self.btnSellAll_clicked)
        nH += 50
 
 
        btnExit = QPushButton('종료', self)
        btnExit.move(20, nH)
        btnExit.clicked.connect(self.btnExit_clicked)
        nH += 50
 
        self.setGeometry(300, 300, 300, nH)
 
    def btnSellAll_clicked(self):
        # 1. 잔고 요청
        self.objConclusion.Unsubscribe()
        if self.obj6033.requestJango(self) == False:
            return
        self.objConclusion.Subscribe('', self)
 
        for code, value in self.jangoData.items():
            print(code, value)
            # 2. 현재가 통신
            self.objRpCur.Request(code)
 
            # 3. 매수 5호가로 매도 주문
            if (self.curData[code]['bid5'] > 0) :
                jangoNum = value['잔고수량']
                amount = value['매도가능']
                if (jangoNum != amount):
                    print("경고: 미체결 수량이 있어 잔고와 매도 가능 수량이 다름", code, jangoNum, amount)
 
                price = self.curData[code]['bid5']
                self.objRpOrder.sellOrder(code, price, amount)
 
 
        return
 
    def btnExit_clicked(self):
        exit()
        return
 
 
    # 매도 후 실시간 주문 체결 받는 로직
    def updateJangoCont(self, pbCont):
        # 주문 체결에서 들어온 신용 구분 값 ==> 잔고 구분값으로 치환
        dicBorrow = {
               '현금': ord(' '),
               '유통융자': ord('Y'),
               '자기융자': ord('Y'),
               '주식담보대출': ord('B'),
               '채권담보대출': ord('B'),
               '매입담보대출': ord('M'),
               '플러스론': ord('P'),
               '자기대용융자': ord('I'),
               '유통대용융자': ord('I'),
               '기타' : ord('Z')
               }
 
        # 잔고 리스트 map 의 key 값
        code = pbCont['종목코드']
 
        # 접수, 거부, 확인 등은 매도 가능 수량만 업데이트 한다.
        if pbCont['체결플래그'] == '접수' or pbCont['체결플래그'] == '거부' or pbCont['체결플래그'] == '확인' :
            if (code not in self.jangoData) :
                return
            self.jangoData[code]['매도가능'] = pbCont['매도가능']
            return
 
        if (pbCont['체결플래그'] == '체결'):
            if (code not in self.jangoData) : # 신규 잔고 추가
                print('신규 잔고 추가', code)
                # 신규 잔고 추가
                item = {}
                item['종목코드'] = pbCont['종목코드']
                item['종목명'] = pbCont['종목명']
                item['현금신용'] = dicBorrow[pbCont['현금신용']]
                item['대출일'] = pbCont['대출일']
                item['잔고수량'] = pbCont['체결기준잔고수량']
                item['매도가능'] = pbCont['매도가능']
                item['장부가'] = pbCont['장부가']
                # 매입금액 = 장부가 * 잔고수량
                item['매입금액'] = item['장부가'] * item['잔고수량']
 
                self.jangoData[code] = item
 
            else:
                # 기존 잔고 업데이트
                item =self.jangoData[code]
                item['종목코드'] = pbCont['종목코드']
                item['종목명'] = pbCont['종목명']
                item['현금신용'] = dicBorrow[pbCont['현금신용']]
                item['대출일'] = pbCont['대출일']
                item['잔고수량'] = pbCont['체결기준잔고수량']
                item['매도가능'] = pbCont['매도가능']
                item['장부가'] = pbCont['장부가']
                # 매입금액 = 장부가 * 잔고수량
                item['매입금액'] = item['장부가'] * item['잔고수량']
 
                # 잔고 수량이 0 이면 잔고 제거
                if item['잔고수량'] == 0:
                    del self.jangoData[code]
                    print('매도 전부 체결', item['종목명'])
 
        return
 
 
 
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()

정상: 관리자권한으로 실행된 프로세스입니다.
777011891 10
777011891 10
통신상태 0 97007모의투자 데이터가 없습니다.                                                     (cii.ast.astevl.selcnsbasestkevl)
0
통신상태 0 97007모의투자 데이터가 없습니다.                                                     (cii.ast.astevl.selcnsbasestkevl)
0


In [ ]:
# Made by Minsuk Sung
# Contact: mssung94@gmail.com
# Homepage: minsuksung-ai.tistory.com
import os
import ctypes
import win32com.client 
# from pywinauto import application
from datetime import datetime
import time

class Daishin:
    
    def __init__(self,msg_on):
        print('대신증권 API 초기화')
        
        self.objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
        # self.objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')  # 코스닥, 코스피 관련
        self.objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
        self.objRq = win32com.client.Dispatch("CpTrade.CpTd6033")  # 주식 잔고 조회 관련
        self.objStockMst = win32com.client.Dispatch("DsCbo1.StockMst")  # 주식 현재가 조회 관련
        self.objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")  # 주식 매수/매도 주문 관련
        self.objStockWeek = win32com.client.Dispatch("DsCbo1.StockWeek")  # 일별 데이터
        self.objStockCur = win32com.client.Dispatch("DsCbo1.StockCur")
        self.objCpCash = win32com.client.Dispatch("CpTrade.CpTdNew5331A")

        # self.slack = Slack()
        # self.today = datetime.now()
        # self.channel_list = {'TEST': '#test'}
        # self.orderType = {'1':'매도', '2':'매수'}
        # self.msg_on = msg_on

        # 프로세스가 관리자 권한으로 실행 여부
        if ctypes.windll.shell32.IsUserAnAdmin():
            print('정상: 관리자권한으로 실행된 프로세스입니다.')
        else:
            print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
            exit(0)

        # 연결 여부 체크
        if self.objCpStatus.IsConnect == 0:
            print("CREON PLUS가 정상적으로 연결되지 않음. ")
            exit(0)

        # 주문 관련 초기화
        if self.objCpTrade.TradeInit(0) != 0:
            print("주문 초기화 실패")
            exit(0)

        self.acc_no = self.objCpTrade.AccountNumber[0]  # 나의 계좌번호
        self.kospi = self.objCodeMgr.GetStockListByMarket(1)  # 코스피
        # self.kosdaq = self.objCodeMgr.GetStockListByMarket(2)  # 코스닥

        # self.slack.notification(
        #     pretext=f"",
        #     title=f"대신증권 트레이딩 시스템 동작",
        #     fallback=f"대신증권 트레이딩 시스템 동작",  # 미리보기로 볼 수 있는
        #     text=f"[INFO] 현재 시각 {self.today.year}년 {self.today.month}월 {self.today.day}일 {self.today.hour}시 {self.today.minute}분 트레이딩 시스템이 동작 되었습니다.",
        #     channel=self.channel_list['TEST'],
        #     msg_on=self.msg_on
        # )

    def get_connect_state(self):
        return self.objCpStatus.IsConnect

    def get_account_info(self):
        accFlag = self.objCpTrade.GoodsList(self.acc_no, 1)  # 주식상품 구분
        self.objRq.SetInputValue(0, self.acc_no)  # 계좌번호
        self.objRq.SetInputValue(1, accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        self.objRq.SetInputValue(2, 50)  # 요청 건수(최대 50)
        self.objRq.SetInputValue(3, 2)  # 수익률구분코드 - ( "1" : 100% 기준, "2": 0% 기준)

        self.objRq.BlockRequest()  # 이거 안해주면 정보 못 가져옴

        self.objCpCash.SetInputValue(0, self.acc_no)
        self.objCpCash.SetInputValue(1, accFlag[0])
        self.objCpCash.BlockRequest()

        # 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        rqRet = self.objRq.GetDibMsg1()
        # print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False

        res = {}

        res['계좌명'] = self.objRq.GetHeaderValue(0)
        res['결제잔고수량'] = self.objRq.GetHeaderValue(1)
        res['체결잔고수량'] = self.objRq.GetHeaderValue(2)
        res['총 평가금액'] = self.objRq.GetHeaderValue(3)
        res['평가손익'] = self.objRq.GetHeaderValue(4)
        res['대출금액'] = self.objRq.GetHeaderValue(6)
        # res['수신개수'] = self.objRq.GetHeaderValue(7)
        res['수익률'] = self.objRq.GetHeaderValue(8)
        res['D+2 예상예수금'] = self.objRq.GetHeaderValue(9)
        res['총평가 내 대주평가금액'] = self.objRq.GetHeaderValue(10)
        res['총평가 내 잔고평가금액'] = self.objRq.GetHeaderValue(11)
        res['대주금액'] = self.objRq.GetHeaderValue(12)

        res['주문 가능 금액'] = self.objCpCash.GetHeaderValue(9)

        txt = f"""
        01. 현재 결제잔고수량: {res['결제잔고수량']:,}
        02. 체결잔고수량: {res['체결잔고수량']:,}
        03. 총 평가금액: {res['총 평가금액']:,}
        04. 평가손익: {res['평가손익']:,}
        05. 대출금액: {res['대출금액']:,}
        06. 수익률: {res['수익률']:,}
        07. D+2 예상예수금: {res['D+2 예상예수금']:,}
        08. 총평가 내 대주평가금액: {res['총평가 내 대주평가금액']:,}
        09. 총평가 내 잔고평가금액: {res['총평가 내 잔고평가금액']:,}
        10. 대주금액: {res['대주금액']:,}
        """

        # self.slack.notification(
        #     pretext=f"",
        #     title=f"현재 계좌({self.acc_no}) 잔고 평가현황",
        #     fallback=f"현재 계좌({self.acc_no}) 잔고 평가현황",  # 미리보기로 볼 수 있는
        #     text=txt,
        #     channel=self.channel_list['TEST'],
        #     msg_on=self.msg_on
        # )

        return res

    # 2번째 예제 활용
    def get_current_data(self, code):
        # 현재가 객체 구하기
        self.objStockMst.SetInputValue(0, code)  # 종목 코드 - 삼성전자
        self.objStockMst.BlockRequest()

        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objStockMst.GetDibStatus()
        rqRet = self.objStockMst.GetDibMsg1()
        # print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()
            
        # 반환할 값을 위한 딕셔너리
        res = {}

        # 현재가 정보 조회
        res['종목코드'] = self.objStockMst.GetHeaderValue(0)  # 종목코드
        res['종목명'] = self.objStockMst.GetHeaderValue(1)  # 종목명
        res['시간'] = self.objStockMst.GetHeaderValue(4)  # 시간
        res['종가'] = self.objStockMst.GetHeaderValue(11)  # 종가
        res['대비'] = self.objStockMst.GetHeaderValue(12)  # 대비
        res['시가'] = self.objStockMst.GetHeaderValue(13)  # 시가
        res['고가'] = self.objStockMst.GetHeaderValue(14)  # 고가
        res['저가'] = self.objStockMst.GetHeaderValue(15)  # 저가
        res['매도호가'] = self.objStockMst.GetHeaderValue(16)  # 매도호가
        res['매수호가'] = self.objStockMst.GetHeaderValue(17)  # 매수호가
        res['거래량'] = self.objStockMst.GetHeaderValue(18)  # 거래량
        res['거래대금'] = self.objStockMst.GetHeaderValue(19)  # 거래대금

        # 예상 체결관련 정보
        res['예상체결가 구분 플래그'] = self.objStockMst.GetHeaderValue(58)  # 예상체결가 구분 플래그
        res['예상체결가'] = self.objStockMst.GetHeaderValue(55)  # 예상체결가
        res['예상체결가 전일대비'] = self.objStockMst.GetHeaderValue(56)  # 예상체결가 전일대비
        res['예상체결수량'] = self.objStockMst.GetHeaderValue(57)  # 예상체결수량

        # print("코드", res['종목코드'])
        # print("이름", res['종목명'])
        # print("시간", res['시간'])
        # print("종가", res['종가'])
        # print("대비", res['대비'])
        # print("시가", res['시가'])
        # print("고가", res['고가'])
        # print("저가", res['저가'])
        # print("매도호가", res['매도호가'])
        # print("매수호가", res['매수호가'])
        # print("거래량", res['거래량'])
        # print("거래대금", res['거래대금'])
        #
        # if (res['예상체결가 구분 플래그'] == ord('0')):
        #     print("장 구분값: 동시호가와 장중 이외의 시간")
        # elif (res['예상체결가 구분 플래그'] == ord('1')):
        #     print("장 구분값: 동시호가 시간")
        # elif (res['예상체결가 구분 플래그'] == ord('2')):
        #     print("장 구분값: 장중 또는 장종료")

        # print("예상체결가 대비 수량")
        # print("예상체결가", res['예상체결가'])
        # print("예상체결가 대비", res['예상체결가 전일대비'])
        # print("예상체결수량", res['예상체결수량'])

        return res

    def get_daily_data(self, code, cnt):
        self.objStockWeek.SetInputValue(0, code)  # 종목 코드 - 삼성전자

        res = {}

        # 최초 데이터 요청
        # 데이터 요청
        self.objStockWeek.BlockRequest()

        # 통신 결과 확인
        ret = None
        rqStatus = self.objStockWeek.GetDibStatus()
        rqRet = self.objStockWeek.GetDibMsg1()
        # print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            ret = False

        # 일자별 정보 데이터 처리
        count = self.objStockWeek.GetHeaderValue(1)  # 데이터 개수
        for i in range(count):
            date = self.objStockWeek.GetDataValue(0, i)  # 일자
            open = self.objStockWeek.GetDataValue(1, i)  # 시가
            high = self.objStockWeek.GetDataValue(2, i)  # 고가
            low = self.objStockWeek.GetDataValue(3, i)  # 저가
            close = self.objStockWeek.GetDataValue(4, i)  # 종가
            diff = self.objStockWeek.GetDataValue(5, i)  # 종가
            vol = self.objStockWeek.GetDataValue(6, i)  # 종가
            res[date] = [open, high, low, close, diff, vol]

        if ret == False:
            exit()

        ret = True

        # 연속 데이터 요청
        # 예제는 5번만 연속 통신 하도록 함.
        NextCount = 1
        while self.objStockWeek.Continue:  # 연속 조회처리
            NextCount += 1;
            if (NextCount > cnt):
                break
            # ret = RequestData(self.objStockWeek)
            rqStatus = self.objStockWeek.GetDibStatus()
            rqRet = self.objStockWeek.GetDibMsg1()
            # print("통신상태", rqStatus, rqRet)
            if rqStatus != 0:
                ret = False

            # 일자별 정보 데이터 처리
            count = self.objStockWeek.GetHeaderValue(1)  # 데이터 개수
            for i in range(count):
                date = self.objStockWeek.GetDataValue(0, i)  # 일자
                open = self.objStockWeek.GetDataValue(1, i)  # 시가
                high = self.objStockWeek.GetDataValue(2, i)  # 고가
                low = self.objStockWeek.GetDataValue(3, i)  # 저가
                close = self.objStockWeek.GetDataValue(4, i)  # 종가
                diff = self.objStockWeek.GetDataValue(5, i)  # 종가
                vol = self.objStockWeek.GetDataValue(6, i)  # 종가
                res[date] = [open, high, low, close, diff, vol]

            if ret == False:
                exit()

        return  res

    def get_my_stocks(self):
        acc = self.acc_no
        accFlag = self.objCpTrade.GoodsList(acc, 1)  # 주식상품 구분

        self.objRq.SetInputValue(0, acc)  # 계좌번호
        self.objRq.SetInputValue(1, accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        self.objRq.SetInputValue(2, 50)  # 요청 건수(최대 50)

        self.objRq.BlockRequest()

        # 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        rqRet = self.objRq.GetDibMsg1()
        # print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False

        cnt = self.objRq.GetHeaderValue(7)
        # print(cnt)

# shcode_list = ['A033780', 'A064350', 'A017670', 'A005490', 'A014680', 'A012450', 'A051900', 'A009150', 'A066570', 'A108320', 'A047810', 'A000660', 'A010140', 'A035420', 'A005380', 'A009830', 'A052690', 'A034020', 'A005070', 'A005420', 'A042700', 'A028050', 'A068270', 'A137310', 'A005930', 'A017960', 'A075580', 'A003490']

        res = {}
        txt = ''
        txt = txt + f"-" * 100 + f"\n"
        txt = txt + f"종목코드 \t 종목명 \t 체결잔고수량 \t 체결장부단가 \t 평가금액 \t 평가손익 \n"

        # print("종목코드 종목명 체결잔고수량 체결장부단가 평가금액 평가손익")
        for i in range(cnt):
            item = {}
            code = self.objRq.GetDataValue(12, i)  # 종목코드
            item['종목명'] = self.objRq.GetDataValue(0, i)  # 종목명
            # retcode.append(code)
            # if len(retcode) >= 200:  # 최대 200 종목만,
            #    break
            item['신용구분'] = self.objRq.GetDataValue(1, i)  # 신용구분
            item['대출일'] = self.objRq.GetDataValue(2, i)  # 대출일
            item['체결잔고수량'] = self.objRq.GetDataValue(7, i)  # 체결잔고수량
            item['체결장부단가'] = self.objRq.GetDataValue(17, i)  # 체결장부단가
            item['평가금액'] = self.objRq.GetDataValue(9, i)  # 평가금액(천원미만은 절사 됨)
            item['수익률'] = self.objRq.GetDataValue(11, i)  # 평가손익
            item['평가손익'] = self.objRq.GetDataValue(10, i)  # 평가손익

            # print(code, item['종목명'], item['대출일'], item['체결잔고수량'], item['체결장부단가'], item['체결장부단가'], item['평가금액'], item['평가손익'])
            txt = txt + f"-" * 100 + f"\n"
            txt = txt + f"{code} \t {item['종목명']} \t {item['체결잔고수량']} \t {item['체결장부단가']} \t {item['평가금액']} \t {item['평가손익']:.4f} \n"
            res[code] = item

        # self.slack.notification(
        #     pretext="",
        #     title=f"현재 계좌({self.acc_no}) 보유 종목 현황",
        #     fallback=f"현재 계좌({self.acc_no}) 보유 종목 현황",  # 미리보기로 볼 수 있는
        #     text=txt,
        #     channel=self.channel_list['TEST'],
        #     msg_on=self.msg_on
        # )

        return res

    def sendOrder(self, bs_type, code, volume):
        # 주식 매수 주문
        acc = self.objCpTrade.AccountNumber[0]  # 계좌번호
        accFlag = self.objCpTrade.GoodsList(acc, 1)  # 주식상품 구분
        # print(acc, accFlag[0])
        self.objStockOrder.SetInputValue(0, bs_type)  # 1: 매도 / 2: 매수
        self.objStockOrder.SetInputValue(1, acc)  # 계좌번호
        self.objStockOrder.SetInputValue(2, accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        self.objStockOrder.SetInputValue(3, code)  # 종목코드 - A003540 - 대신증권 종목
        self.objStockOrder.SetInputValue(4, volume)  # 매수수량 10주
        # self.objStockOrder.SetInputValue(5, price)  # 주문단가  - 14,100원 시장가 매수/매도의 경우 가격 의미 없음
        self.objStockOrder.SetInputValue(7, "0")  # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
        self.objStockOrder.SetInputValue(8, "03")  # 주문호가 구분코드 - 01: 보통 / 02: 임의 / 03: 시장가

        # 매수 주문 요청
        self.objStockOrder.BlockRequest()

        rqStatus = self.objStockOrder.GetDibStatus()
        rqRet = self.objStockOrder.GetDibMsg1()
        # print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()

        # self.slack.notification(
        #     pretext="",
        #     title=f"{code} {volume}주 {self.orderType[bs_type]} 완료",
        #     fallback=f"{code} {volume}주 {self.orderType[bs_type]} 완료",  # 미리보기로 볼 수 있는
        #     text=f"{rqRet}",
        #     channel=self.channel_list['TEST'],
        #     msg_on=self.msg_on
        # )


if __name__ == '__main__':

    import pandas as pd

    daishin = Daishin(msg_on=False)
    # print(daishin.acc_no)
    # print(f"현재 연결된 계좌번호: {daishin.acc_no}")
    # print(f"현재 코스피 종목수: {len(daishin.kospi)}")
    # print(f"현재 코스닥 종목수: {len(daishin.kosdaq)}")
    # print(f"삼성전자의 현재가 정보: {daishin.get_current_data('A005930')}")
    # print(pd.DataFrame(daishin.get_daily_data('A005930',5)).T)
    # daishin.get_account_info()
    # daishin.get_my_stocks()
    # daishin.sendOrder("2", "035420", 5)
    print(daishin.kospi)

대신증권 API 초기화
정상: 관리자권한으로 실행된 프로세스입니다.
('A000020', 'A000040', 'A000050', 'A000070', 'A000075', 'A000080', 'A000087', 'A000100', 'A000105', 'A000120', 'A000140', 'A000145', 'A000150', 'A000155', 'A000157', 'A000180', 'A000210', 'A000215', 'A000220', 'A000225', 'A000227', 'A000230', 'A000240', 'A000270', 'A000300', 'A000320', 'A000325', 'A000370', 'A000390', 'A000400', 'A000430', 'A000480', 'A000490', 'A000500', 'A000520', 'A000540', 'A000545', 'A000590', 'A000640', 'A000650', 'A000660', 'A000670', 'A000680', 'A000700', 'A000720', 'A000725', 'A000760', 'A000810', 'A000815', 'A000850', 'A000860', 'A000880', 'A000885', 'A00088K', 'A000890', 'A000910', 'A000950', 'A000970', 'A000990', 'A001020', 'A001040', 'A001045', 'A00104K', 'A001060', 'A001065', 'A001067', 'A001070', 'A001080', 'A001120', 'A001130', 'A001140', 'A001200', 'A001210', 'A001230', 'A001250', 'A001260', 'A001270', 'A001275', 'A001290', 'A001340', 'A001360', 'A001380', 'A001390', 'A001420', 'A001430', 'A001440', 'A001450', 'A00

: 

: 